# Import Libs & Data

In [35]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')

In [37]:
print(concrete_data.columns.values)

['Cement' 'Blast Furnace Slag' 'Fly Ash' 'Water' 'Superplasticizer'
 'Coarse Aggregate' 'Fine Aggregate' 'Age' 'Strength']


In [38]:
# We can easily check the dataset for any missing values by using the following one-liner:  
concrete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [39]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# Data Analysis 

In [40]:
# Remove the results from the dataset (which is the target data)
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] 
# The target data is the result
target = concrete_data['Strength'] 

In [41]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [42]:
target.head() # The target data "strength"

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [43]:
# Also we can normalize the data by substracting the mean and dividing by the standard deviation.

# Z = (X - μ) / σ, its a standard score of the sample (Gaussian distribution)
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [44]:
# Let's save the number of predictors to n_cols since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

# Model Build

In [45]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=0)

# Display the shapes of the resulting datasets
print(f"Training predictors shape: {X_train.shape}")
print(f"Test predictors shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test target shape: {y_test.shape}")

Training predictors shape: (721, 8)
Test predictors shape: (309, 8)
Training target shape: (721,)
Test target shape: (309,)


In [46]:
# Keras model

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [47]:
# build the model
model = regression_model()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
# fit the model using only the training data
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)

Epoch 1/50


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1696.3888 - val_loss: 1480.3903
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1524.4229 - val_loss: 1469.5571
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1547.5820 - val_loss: 1458.5145
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1522.2335 - val_loss: 1447.6608
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1578.4163 - val_loss: 1436.7323
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1507.8829 - val_loss: 1425.8761
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1552.4580 - val_loss: 1415.0656
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1508.9702 - val_loss: 1404.1495
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1527.3987 - val_loss: 1393.3508
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1445.0972 - val_loss: 1382.1357
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1495.4207 - val_loss: 1370.5792
Epoch 12/50
16/16 ━

# Predict the results

In [49]:
# Predict the results for the test set
y_pred = model.predict(X_test)

# Display the predicted results
print(y_pred)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[15.569835 ]
 [30.735756 ]
 [42.642963 ]
 [30.832344 ]
 [ 3.9589345]
 [15.094736 ]
 [ 5.2131996]
 [12.82655  ]
 [ 8.436335 ]
 [ 5.864352 ]
 [ 5.671614 ]
 [ 5.4782863]
 [51.15412  ]
 [36.65047  ]
 [ 9.256469 ]
 [26.524881 ]
 [ 4.9396806]
 [11.758218 ]
 [12.929877 ]
 [ 7.631192 ]
 [12.244297 ]
 [11.628854 ]
 [ 5.7651844]
 [12.961887 ]
 [10.137965 ]
 [ 8.327241 ]
 [ 7.1738124]
 [20.66489  ]
 [14.346364 ]
 [ 5.0082803]
 [ 6.216615 ]
 [ 7.486446 ]
 [19.935036 ]
 [15.754737 ]
 [ 7.3963842]
 [17.613123 ]
 [ 8.379542 ]
 [15.263301 ]
 [ 6.8387847]
 [ 7.5910583]
 [ 5.0260158]
 [ 4.745739 ]
 [ 4.7319794]
 [15.116367 ]
 [ 4.914864 ]
 [28.961983 ]
 [20.331505 ]
 [13.448625 ]
 [ 3.1737318]
 [ 8.236809 ]
 [20.011118 ]
 [ 7.098292 ]
 [11.205611 ]
 [15.040742 ]
 [10.093566 ]
 [30.241743 ]
 [ 7.8685164]
 [ 5.841666 ]
 [11.332405 ]
 [ 6.2678223]
 [14.889494 ]
 [ 7.6207337]
 [ 7.3145785]
 [33.368813 ]
 [ 6.2421002]
 [ 7.896625 ]
 [ 9.752743 ]
 [ 5.738098 ]
 [ 5.239

In [50]:
# Compute the mean squared error between the predicted and actual concrete strength
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 702.9296380384438


# For the remaining 50-1 = 49 loops

In [51]:
# Initialize a list to store the mean squared errors
mse_list = [mse]

# Repeat the process 50 times
for i in range(1,50):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    # Build the model
    model = regression_model()
    
    # Fit the model using only the training data
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    
    # Predict the results for the test set
    y_pred = model.predict(X_test)
    
    # Compute the mean squared error between the predicted and actual concrete strength
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the mse to the list
    mse_list.append(mse)

# Calculate the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Mean of Mean Squared Errors: 639.4645602720059
Standard Deviation of Mean Squared Errors: 113.05932284527535
